<a href="https://colab.research.google.com/github/Imran0897/Placement-Assignment_Md_Imran/blob/main/DL_03_Placement_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TensorFlow Implementation:

In [42]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype("float32") / 255.0

# Define the Pure CNN model
model = Sequential([
    Conv2D(8, (5, 5), padding="same", activation="relu", input_shape=(28, 28, 1)),
    MaxPooling2D(4, 4),
    Conv2D(12, (5, 5), padding="same", activation="relu"),
    Flatten(),
    Dense(10, activation="softmax")
])

# Print the model summary
model.summary()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=["accuracy"])

# Train the model
num_epochs = 10
batch_size = 64
min_val_accuracy = 99.4
best_model_path = "best_model.h5"
best_val_accuracy = 0.0

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode="max")

model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stopping])

# Save the best model based on validation accuracy
model.save(best_model_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 8)         208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 7, 8)          0         
 )                                                               
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 12)          2412      
                                                                 
 flatten_3 (Flatten)         (None, 588)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5890      
                                                                 
Total params: 8,510
Trainable params: 8,510
Non-trainable params: 0
____________________________________________________

PyTorch Implementation:

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchsummary import summary

# Set device configuration (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the Pure CNN model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8, 12, 3, padding=1)
        self.conv3 = nn.Conv2d(12, 16, 3, padding=1)
        self.fc = nn.Linear(16 * 7 * 7, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 16 * 7 * 7)
        x = self.fc(x)
        return x

# Create the Pure CNN model
model = PureCNN().to(device)

# Print the model summary
summary(model, input_size=(1, 28, 28))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total * 100
    return accuracy

# Train the model
num_epochs = 10
min_val_accuracy = 99.4
best_model_path = "best_model.pth"
best_val_accuracy = 0.0


for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)

    # Calculate average loss and accuracy for the epoch
    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)

    # Evaluate the model on the validation set
    model.eval()
    val_accuracy = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)

    val_accuracy /= len(test_loader.dataset)

    # Save the model with the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Accuracy: {val_accuracy:.2f}%")

    # Stop training if the desired validation accuracy is reached
    if val_accuracy >= min_val_accuracy:
        print("Desired validation accuracy reached. Training stopped.")
        break

# Load the best model and evaluate it on the test set
model.load_state_dict(torch.load(best_model_path))
model.eval()

test_accuracy = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)

test_accuracy /= len(test_loader.dataset)

print(f"Test Accuracy: {test_accuracy:.2f}%")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
         MaxPool2d-2            [-1, 8, 14, 14]               0
            Conv2d-3           [-1, 12, 14, 14]             876
         MaxPool2d-4             [-1, 12, 7, 7]               0
            Conv2d-5             [-1, 16, 7, 7]           1,744
            Linear-6                   [-1, 10]           7,850
Total params: 10,550
Trainable params: 10,550
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.04
Estimated Total Size (MB): 0.13
----------------------------------------------------------------
Epoch [1/10] - Train Loss: 0.3396, Train Accuracy: 89.74%, Validation Accuracy: 96.73%
Epoch [2/10] - Train Loss: 0.0905, Train Accuracy: 97.28%, Validation Accuracy: 98.18